In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random

from networkx.algorithms import bipartite

In [2]:
"""
Definitions
"""
random.seed('Zufall!!')
mapWidth = mapHeight = 250

### Skills
Im Folgenden wird die Klasse _Skills_ implementiert. Sie repräsentiert einen Skill mit definierten Namen. Wird kein Name zur Initialisierung übergeben, wird ein Namen aus der ID (fortlaufende Nummer über alle initialisierten Skills) generiert.
Danach werden beispielhaft vier Skills initialisiert. 

In [3]:
class Skill:
    count = 0
    
    def __init__(self, name=None):
        self._id = Skill.count
        Skill.count += 1
        if name is None:
            self.name = "Skill " + str(self._id)
        else:
            self.name = name
            
            
    def __str__(self):
        return self.name

In [4]:
sBaggern = Skill("baggern"); sHeben = Skill("heben")
sMischen = Skill("mischen"); sMauern = Skill("mauern")

skills = [sBaggern, sHeben, sMischen, sMauern]
print "Skills: " + ", ".join([str(skill) for skill in skills])

Skills: baggern, heben, mischen, mauern


## Skillkapazitäten und Agenten (Baufirma)
### Skillkapazität
Agenten (Baufirmen) bieten Skills nicht einzeln, sondern als Skillkapazität an (in einer Skillkapazität kann jedoch auch ein Skill nur einmal vorhanden sein). Die Kosten des Agentens für die Bereitstellung der kompletten Skillkapazität oder eines Teiles werden über eine spezifische Kostenfunktion berechnet. Diese wird bei der Initialisierung der Klasse übergeben und rechnet auf Grundlage der Kapazität des Skills und der Entfernung zwischen Agent und Baustelle. Für ein Beispiel initialisieren wir uns 2 verschiedenen Skillkapazitäten mit unterschiedlicher Kostenfunktion.

In [5]:
class SkillCap:
    count = 0
    
    def __init__(self, skill, capacity, costFunction):
        """ Initialisiere eine Skillkapazität mit Skill, Menge und Kostenfunktion. """
        
        self._id = SkillCap.count
        SkillCap.count += 1
        
        self.skill = skill
        self.capacity = capacity
        self.costFunction = costFunction
        
    def __str__(self):
        return "C-" + str(self.skill) + "(" + str(self.capacity) + ")"
    
    
    def costs(self, distance, count=None):
        """ Kosten für eine Menge der Skillkapazität. Wenn keine Menge angegeben wird: Kosten
            für die gesamten Skillkapazität. """
    
        if count is None:
            count = self.capacity
        
        if  0 < count and count <= self.capacity:
            return self.costFunction(count, distance)
        else:
            raise Exception("SkillCapacity", "getCosts()")
    
    
    def sell(self, distance, count=None):
        """ Verkaufen eines Teiles oder der gesamten Skillkapazität (wenn keine Anzahl angegeben wird). 
            Die Kosten für den Agenten nach der Kostenfunktion werden zurückgegeben. """ 
        
        if count is None:
            count = self.capacity
        
        if 0 < count and count <= self.capacity:
            self.capacity -= count
            return self.costs(distance, count)
        else:
            raise StandardException("SkillCapacity", "sell()")

In [6]:
sk1 = SkillCap(sBaggern, 10, (lambda x,y: (x/3+1)*y + 200*x))  # 10 items, 3 items per shipment, one item 200
sk2 = SkillCap(sMauern, 8, (lambda x,y: (x/3+1)*y + 215*x))  # 8 items, 3 items per shipment, one item 215
skillCaps = [sk1, sk2]; print "Skillkapazitäten: " + ", ".join([str(skillCap) for skillCap in skillCaps])

print "\nKosten für 2 Items sk1, Distanz 5: " + str(sk1.costs(5, 2))
print "Kosten für 4 Items sk1, Distanz 5: " + str(sk1.costs(5, 4))

sk1.sell(5, 2); sk1.sell(5, 4)
print "sk1 nach dem Verkauf der Items: " + str(sk1)

Skillkapazitäten: C-baggern(10), C-mauern(8)

Kosten für 2 Items sk1, Distanz 5: 405
Kosten für 4 Items sk1, Distanz 5: 810
sk1 nach dem Verkauf der Items: C-baggern(4)


### Agent (Baufirma)
Eine Agent (Baufirma) in unserem Spiel sitzt an einem bestimmten Punkt im Koordinatensystem und hält verschiedenen Skillkapazitäten bereit. Im Laufe des Spiel verkauft er diese an Baustellen.  

In [7]:
class Agent:
    count = 0
    
    def __init__(self, skillCaps, x=(random.uniform(0, mapWidth)), y=(random.uniform(0, mapHeight)) ):
        """ Initialisieren eines Agenten mit übergebenen Skillkapazitäten. Werden keine Koordinaten übergeben, 
            wird der Agent zufällig auf der Karte platziert. """
        
        self._id = "A"+str(Agent.count)
        Agent.count += 1
        
        self.x, self.y = x, y
        self.skillCaps = skillCaps
    
    
    def __str__(self):
        skillString = ""
        if len(self.skillCaps) > 0:
            skillString = ": " + ", ".join([str(skillCap) for skillCap in self.skillCaps])
        
        return("Agent " + str(self._id) + skillString)
    
    @property
    def supply(self):  # Agent, Skill, Menge
        return np.array([(self._id, str(skillCap.skill), str(skillCap.capacity)) for skillCap in self.skillCaps])

In [8]:
sk1 = SkillCap(sBaggern, 10, (lambda x,y: (x/3+1)*y + 200*x))  # 10 items, 3 items per shipment, one item 200
sk2 = SkillCap(sMauern, 8, (lambda x,y: (x/3+1)*y + 215*x))  # 8 items, 3 items per shipment, one item 215

agent1 = Agent((sk1, sk2))
print agent1

Agent A0: C-baggern(10), C-mauern(8)


## Ausschreibungen und Baustellen
### Ausschreibung
Eine Ausschreibung verrät uns, welchen Skill und in welcher Menge dieser von einer Baustelle für die erfolgreiche Fertigstellung benötigt wird. 

In [9]:
class SkillReq:
    count = 0
    
    def __init__(self, skill, amount):
        """ Initialisiere einer Ausschreibung mit Skill und Menge. """
    
        self._id = SkillReq.count
        SkillReq.count += 1
        
        self.skill = skill
        self.amount = amount
    
    def __str__(self):
        return "R-" + str(self.skill) + "(" + str(self.amount) + ")"
    
    def buy(self, count=None):
        """ Kaufen einer Menge des Skills: Der Request wird entsprechend angepasst. """
        
        if count is None:
            count = self.amount
        
        if  0 < count and count <= self.amount:
            self.amount -= count
        else:
            raise StandardException("Request", "buy()")

In [10]:
rq1 = SkillReq(sBaggern, 5); rq2 = SkillReq(sBaggern, 5)
rq3 = SkillReq(sMauern, 3); rq4 = SkillReq(sMauern, 2)

requests = [rq1, rq2, rq3, rq3]
print "Requests: " + ", ".join([str(request) for request in requests])

Requests: R-baggern(5), R-baggern(5), R-mauern(3), R-mauern(3)


### Baustelle
Im Laufe des Spiel sind es die Baustellen, deren Ausschreibungen im Rahmen ihres Budgets möglichst preiswert von den Agenten erfüllt werden sollen. Auch sie sind fest im Koordinatensystem einem Ort zugeordnet.

In [11]:
class Site:
    count = 0
    
    def __init__(self, requests, budget, x=(random.uniform(0, mapWidth)), y=(random.uniform(0, mapHeight)) ):
        """ Initialisiere einer Baustelle mit übergebenen Request. Werden keine Koordinaten übergeben, 
            wird die Baustelle zufällig auf der Karte platziert. """
        
        self._id = "B"+str(Site.count)
        Site.count += 1
        
        self.x, self.y = x, y
        self.requests = requests
        self.bud = budget
    
    def __str__(self):
        requestString = ""
        if len(self.requests) > 0:
            requestString = ": " + ", ".join([str(request) for request in self.requests])
        
        return("Baustelle " + str(self._id) + requestString)
    
    @property
    def demand(self):  # Baustelle, Skill, Menge
        return np.array([(self._id, request.skill.name, request.amount) for request in self.requests])
    
    @property
    def budget(self):  # Baustelle, Budget
        return [self._id, self.bud]

In [12]:
site1 = Site([rq1, rq3], 2000); site2 = Site([rq2, rq4], 2500)
print site1; print site2

Baustelle B0: R-baggern(5), R-mauern(3)
Baustelle B1: R-baggern(5), R-mauern(2)


## Szenario

In [13]:
class Scenario:
    
    
    def __init__(self, agentCount=2, siteCount=2, skills=[Skill("mauern"), Skill("baggern")], \
                 skillCap=[[10,20],[15,10]], skillReq=[[7,15],[8,8]], budget=[100000], \
                 costFunction=[[lambda x,y:(x/5+1)*y+200*x, lambda x,y:(x/3+1)*y+250*x]], \
                 agents=None, sites=None):
        """ Initialisierung eines Szenarios: Ohne die weitere Angabe von Parametern wird ein Szenario mit zwei Agenten 
            und einer Baustelle erstellt, die zwei Skills ("mauern" und "baggern") in der Menge (10,20) für Agent 1 
            und (15,10) für Agent 2 als Skillkapazität besitzen bzw. Baustelle 1 (7,15) und Baustelle 2 (8,8) suchen.
            Es gibt für die Skillkapazitäten Standardkostenfunktionen, die z.B. für den ersten Skill von Kosten von 
            200 per item und Shipmentkosten von $distance je 5 items ausgeht. Von dem dritten Skill könne nur 3 auf 
            einmal transportiert werden und eine Einheit kostet 250. Das Defaultbudget der Baustellen beträgt 100.000. 
            Alternativ hätte auch für jeden Agenten eine eigene Kostenfunktion bzw. jede Baustelle ein eigenes Budget 
            definiert werden können. Wenn Baustellen und Agenten bereits der Funktion übergeben werden (als Iterable),
            so werden diese benutzt. """
        
        # Are there complete sites? ############
        if sites is not None:
            self.sites = sites
        else:
            # check if budget sitewise or for all sites?
            if len(budget) == 1:
                budget = budget * siteCount
            elif len(budget) != siteCount:
                raise StandardException("Scenario", "__init__()", "site-budget")
            
            # check skill requests
            if len(skillReq) == 1:
                skillReq = [skillReq] * siteCount
            elif len(skillReq) != siteCount:
                raise StandardException("Scenario", "__init__()", "site-requests")
                
            # create sites
            self.sites = list()
            for site in range(siteCount):
                
                # create Skill Requests
                sk = list()
                if len(skillReq[site]) == len(skills):
                    for i, skill in enumerate(skills):
                        sk.append(SkillReq(skill, skillReq[site][i]))
                elif len(skillReq) != siteCount:
                    raise StandardException("Scenario", "__init__()", "site-requests-creation")

                self.sites.append(Site(sk, budget[site]))
        
        # Are there complete agents? ############
        if agents is not None:
            self.agents = agents
        else:
            # check if costFunction sitewise or for all sites?
            if len(costFunction) == 1:
                costFunction = costFunction * agentCount
            elif len(costFunction) != agentCount:
                raise StandardException("Scenario", "__init__()", "agent-costFunction")
            
            # check skill capacities
            if len(skillCap) == 1:
                skillCap = [skillCap] * agentCount
            elif len(skillCap) != agentCount:
                raise StandardException("Scenario", "__init__()", "aget-capabilities")
                
            # create agents
            self.agents = list()
            for agent in range(agentCount):
                
                # create Skill Capacities
                sk = list()
                if len(skillCap[agent]) == len(skills):
                    for i, skill in enumerate(skills):
                        sk.append(SkillCap(skill, skillCap[agent][i], costFunction[agent][i]))
                elif len(skillReq) != siteCount:
                    raise StandardException("Scenario", "__init__()", "agent-cap-creation")
                    
                self.agents.append(Agent(sk))
     
        
    def __str__(self):
        """ Ausgabe des Szenarios in der definierten Algebra-Schreibweise. """
        
        supply = [ ", ".join([elem[0], elem[1], elem[2]]) for elem in list(self.supply)]
        demand = [ ", ".join([elem[0], elem[1], elem[2]]) for elem in list(self.demand)]
        budget = [ ", ".join([elem[0], elem[1]]) for elem in list(self.budget)]
        
        return "Agenten: " + ", ".join(self.agent) + "\nBaustellen: " + ", ".join(self.site) + "\n" + \
               "Supply: " +  "("+"), (".join(supply)+")" + "\nDemand: " +"("+"), (".join(demand)+")\n" + \
               "Budget: " + "("+"), (".join(budget)+")"
    
    @property
    def agent(self):  # Agent
        return np.array([agent._id for agent in self.agents])
    
    @property
    def site(self):  # Baustelle
        return np.array([site._id for site in self.sites])
                                           
    @property
    def supply(self):  # Agent, Skill, Menge
        return np.concatenate([agent.supply for agent in self.agents], axis=0)
    
    @property
    def demand(self):  # Baustelle, Skill, Menge
        return np.concatenate([site.demand for site in self.sites], axis=0)
    
    @property
    def budget(self):  # Baustelle, Budget
        return np.array([site.budget for site in self.sites])
    
    
    #def match(self, agent, site, skillCap, request, price, amount=-1):
        """ Ein Matching/Verkaufsvorgang wird beschrieben durch den behalten Preis durch die Baustelle, 
            den Kosten des Agenten, den Skill, die Kapazität und die Baustelle und den Agent selbst. """
    #    cost = skillCap.sell(self.distance(agent, site), amount)
    #    request.buy(amount)
    #    np.vstack([self.sales, (agent._id, site._id, str(skillCap.skill), cost, price))
    

    def distance(a, b):
        return ( ((a.x-b.x)**2) + ((a.y-b.y)**2) )**0.5

In [14]:
sc1 = Scenario()
print sc1

Agenten: A1, A2
Baustellen: B2, B3
Supply: (A1, mauern, 10), (A1, baggern, 20), (A2, mauern, 15), (A2, baggern, 10)
Demand: (B2, mauern, 7), (B2, baggern, 15), (B3, mauern, 8), (B3, baggern, 8)
Budget: (B2, 100000), (B3, 100000)
